In [23]:

!pip install qgrid

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import h5py
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Attention, Input
from livelossplot import PlotLossesKeras

In [2]:
# Specify the stock symbol and the date range
stock_symbol = "AAPL"
start_date = "2003-01-01"
end_date = "2023-12-18"

# Fetch the stock data
apple_cl = yf.download(stock_symbol, start=start_date, end=end_date)

#apple_cl.set_index("Date", inplace=True)

# Display the stock data
apple_cl.head()
apple_cl.info()

[*********************100%%**********************]  1 of 1 completed
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5276 entries, 2003-01-02 to 2023-12-15
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5276 non-null   float64
 1   High       5276 non-null   float64
 2   Low        5276 non-null   float64
 3   Close      5276 non-null   float64
 4   Adj Close  5276 non-null   float64
 5   Volume     5276 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 288.5 KB


In [3]:
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler

# Specify the stock symbol and the date range
stock_symbol = "AAPL"
start_date = "2003-01-01"
end_date = "2023-12-18"

# Fetch the stock data
apple_cl = yf.download(stock_symbol, start=start_date, end=end_date)

# Display the stock data with the current index
print(apple_cl.head())

# Check if 'Close' column is present
if 'Close' in apple_cl.columns:
    # Preprocess the dataset
    data = apple_cl['Close'].values.reshape(-1, 1)

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 8))
    scaled_data = scaler.fit_transform(data)

    # Continue with your code using 'scaled_data'
    # ...

else:
    print("Error: 'Close' column not found in the DataFrame.")



[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close     Volume
Date                                                                    
2003-01-02  0.256429  0.266429  0.256250  0.264286   0.224030  181428800
2003-01-03  0.264286  0.266607  0.260536  0.266071   0.225543  147453600
2003-01-06  0.268393  0.274643  0.265714  0.266071   0.225543  390532800
2003-01-07  0.264107  0.267857  0.258393  0.265179   0.224787  342344800
2003-01-08  0.260357  0.262679  0.257857  0.259821   0.220245  229644800


In [4]:
print(apple_cl.columns)


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')


In [5]:
# Split the dataset into training and testing sets
train_size = int(0.8 * len(scaled_data))
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [6]:
# Load the saved model
base_model = load_model("model_ttest_v2.h5")

In [7]:
# Print the input and output shapes
input_shape = base_model.input_shape
output_shape = base_model.output_shape

input_length = input_shape[1]
output_length = output_shape[1]

print("Input Length:", input_length)
print("Output Length:", output_length)

Input Length: 1
Output Length: 3


In [8]:
# Display the model architecture
base_model.summary()

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_29 (LSTM)              (None, 150)               95400     
                                                                 
 dense_29 (Dense)            (None, 3)                 453       
                                                                 
Total params: 95853 (374.43 KB)
Trainable params: 95853 (374.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
## Create input and output sequences
input_length = 1
output_length = 3

def create_sequences(data, input_length, output_length):
    X = []
    y = []
    for i in range(len(data) - input_length - output_length + 1):
        X.append(data[i:i + input_length])
        y.append(data[i + input_length:i + input_length + output_length])
    return np.array(X), np.array(y)


train_X, train_y = create_sequences(train_data, input_length, output_length)
test_X, test_y = create_sequences(test_data, input_length, output_length)
print("Before Reshaping:")
print("train_X shape:", train_X.shape)
print("test_X shape:", test_X.shape)

Before Reshaping:
train_X shape: (4217, 1, 1)
test_X shape: (1053, 1, 1)


In [10]:
# Reshape the data
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))

# Print shapes after reshaping
print("After Reshaping:")
print("train_X shape:", train_X.shape)
print("test_X shape:", test_X.shape)

After Reshaping:
train_X shape: (4217, 1, 1)
test_X shape: (1053, 1, 1)


In [11]:
# Freeze the layers
for layer in base_model.layers:
    layer.trainable = False

In [12]:
base_model.trainable = False

In [13]:
# Create the new model by adding layers to the base model
new_model_input = Input(shape=base_model.input_shape[1:])  # Exclude the batch size dimension
output = base_model(new_model_input)

# Add the attention layer
output = Attention()([output, output])

# Add additional layers to the new model
output = Dense(128, activation='relu')(output)
output = Dense(1, activation='relu')(output)

new_model = Model(inputs=new_model_input, outputs=output)

# Display the new_model architecture
new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1, 8)]               0         []                            
                                                                                                  
 sequential_29 (Sequential)  (None, 3)                    95853     ['input_1[0][0]']             
                                                                                                  
 attention (Attention)       (None, 3)                    0         ['sequential_29[0][0]',       
                                                                     'sequential_29[0][0]']       
                                                                                                  
 dense (Dense)               (None, 128)                  512       ['attention[0][0]']       

In [14]:
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))

In [15]:
# Compile the model
new_model.compile(optimizer='adam', loss='mean_squared_error')

In [30]:
## Train the model
initial_epochs = 10
#history = new_model.fit(train_X, train_y, epochs=initial_epochs, batch_size=32, validation_data=(test_X, test_y))
history = new_model.fit(train_X, train_y,epochs=10,validation_data=(test_X, test_y),callbacks=[PlotLossesKeras()],verbose=0)

ValueError: in user code:

    File "C:\Users\chris\Documents\1_spicy\000_datadrivendollars\ds-capstone\.venv\Lib\site-packages\keras\src\engine\training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\chris\Documents\1_spicy\000_datadrivendollars\ds-capstone\.venv\Lib\site-packages\keras\src\engine\training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\chris\Documents\1_spicy\000_datadrivendollars\ds-capstone\.venv\Lib\site-packages\keras\src\engine\training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\chris\Documents\1_spicy\000_datadrivendollars\ds-capstone\.venv\Lib\site-packages\keras\src\engine\training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\chris\Documents\1_spicy\000_datadrivendollars\ds-capstone\.venv\Lib\site-packages\keras\src\utils\traceback_util

In [ ]:
# Evaluate the model on the test set before training
loss0 = new_model.evaluate(test_X, test_y)
print("initial loss::", loss0)

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Mean Squared Error')
#plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 2

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [ ]:
# Unfreeze the last few layers for fine-tuning
unfreeze_layers = 3
for layer in base_model.layers[-unfreeze_layers:]:
    layer.trainable = True

In [ ]:
# Compile the model
new_model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Displt the updated new_model architecture
new_model.summary()

In [ ]:
len(new_model.trainable_variables)

In [ ]:
# Train the model with the fine-tuning process
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = new_model.fit(train_X, train_y, epochs=total_epochs, initial_epoch=history.epoch[-1], batch_size=32, validation_data=(test_X, test_y))

In [ ]:
# visualize the loss curve of the training and validation
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
# plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
# Evaluate the model on the test set after fine-tuning training
loss = new_model.evaluate(test_X, test_y)
print("loss::", loss)

In [ ]:
# Get predictions for the test set
predictions = new_model.predict(test_X)

# Reshape the predictions array
predictions = predictions.reshape(-1, output_length)

# Rescale the predictions and actual values
predictions = scaler.inverse_transform(predictions)
actual_values = scaler.inverse_transform(test_y.reshape(-1, output_length))

# Flatten the arrays for plotting
predictions = predictions.flatten()
actual_values = actual_values.flatten()

# Plot the predictions and actual values
plt.figure(figsize=(10, 6))
plt.plot(predictions, label='Predictions')
plt.plot(actual_values, label='Actual Values')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.title('Predictions vs Actual Values')
plt.legend()
plt.show()